# Assignment 3

\- Yash Gupta (190997)

### Part 1: 25 marks

Fill in the partially complete lines of code in the file temporal_context_model_empty.m to make a temporal context model of memory retrieval that retrieves about 7 items from an encoded list efficiently.

This should not be very challenging if you've followed the concept of the TCM in class. 

In [ ]:
from black import out
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rand
import random

In [ ]:
def draw_from_a_dist(p):
    if np.sum(p) == 0:
        p = 0.05 * np.ones(len(p))
    p /= np.sum(p)
    idx = np.where(rand.random(1) - np.cumsum(p) < 0)
    sample = np.min(idx)
    out = np.zeros(len(p))
    out[sample] = 1
    return out

In [ ]:
N_WORLD_FEATURES = 5
N_ITEMS = 10
ENCODING_TIME = 500
TEST_TIME = 20

In [ ]:
schedule = np.column_stack((np.sort(np.round(rand.random(N_ITEMS) * ENCODING_TIME)), np.arange(N_ITEMS)))
schedule_load = ENCODING_TIME / np.median(np.diff(schedule[:, 0]))
encoding = np.zeros((N_ITEMS, N_WORLD_FEATURES + 1))

world_m = np.array([1, 2, 1, 2, 3], dtype=float)
world_var = 1
delta = 0.05
beta_param = 0.001
m = 0

for time in range(ENCODING_TIME):
    world_m += delta
    world = rand.normal(world_m, world_var)
    # 
    if m < N_ITEMS:
        if time == schedule[m, 0]:
            encoding[m, :] = np.append(world, m)
            m += 1

out = np.zeros(TEST_TIME)
while time < ENCODING_TIME + TEST_TIME:
    world_m += delta
    world = rand.normal(world_m, world_var)
    soa = np.zeros(N_ITEMS)
    for m in range(N_ITEMS):
        soa[m] = np.dot(encoding[m], np.append(world, m))
    soa /= np.linalg.norm(soa)
    out[time - ENCODING_TIME] = np.where(draw_from_a_dist(soa) > 0)[0]
    time += 1

success = len(np.unique(out))

### Part 2: 25 marks

Alongside the TCM you have implemented, you have also implemented assumptions about the physics of the world. Namely, you have assumed that the world can be represented as a set of draws from N independent Gaussians, and that the means of these Gaussians evolve over time linearly. Now we change this. We assume that the world contains context changes, and represent this fact by sampling the rate of drift over time (delta) in the feature means itself from a mixture of two Gaussians, one with a small mean to denote small changes, and one with a large mean to denote large changes. 

Implement this new physics of the world in the model, and design an encoding schedule that lets your model retrieve effectively (>7 success). The trivial solution is to shove all the encodings towards the back end of the encoding period, but doing so will increase your encoding load (proxy for study effort). 

An optimal solution would find the scheduling pattern that minimizes % encoding load while maintaining average retrieval success across multiple runs at 7. See if you can find it, assuming your retrieving agent knows the parameters of the model that will generate his world contexts.

### Part 3: 50 marks

Do the same thing, but this time assuming that the retrieving agent does not know the parameters of the world generating model, just the fact that the rate of drift is sampled from a bimodal distribution. Hint: you'll have to have the agent learn the drift distribution parameters. EM is your friend here. 